In [2]:
import tensorflow as tf
print('TF version: ', tf.__version__)
import glob
import cv2
from IPython import display


import matplotlib.pyplot as plt
import os, time  
import numpy as np 
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

TF version:  2.1.0


In [3]:
import image_helper_for_DCGAN as helper

data_dir = "H://PatternLab//PatternRecognition//Datasets//brain//keras_png_slices_data//keras_png_slices_train//path_test"
tf_X_train = helper.cv_get_images(data_dir)
tf_X_train.shape

print('max = ', tf.reduce_max(tf_X_train).numpy())
print('min = ', tf.reduce_min(tf_X_train).numpy())


max =  0.8901961
min =  -1.0


In [1]:
import DCGAN_utilities as DCGAN

2.1.0
